<a href="https://colab.research.google.com/github/ck1972/University-GeoAI/blob/main/Lab_2C_Mapping_LandCover_S2_RF_Model_GEE_Colombo_Tutorial1_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Mapping Land Cover using Sentinel-2 and a Random Forest Model: An Introductory Guide with Python
## Aim
The script maps land cover using Sentinel-2 (S2), spectral indices, and a random forest method. We use Bulawayo City (Zimbabwe) as the test site.

## Requirements
To run this script, the user must have an Earth Engine account. In addition, the user must authenticate the Earth Engine Python API. See the instructions [here](https://developers.google.com/earth-engine/guides/auth).


This script will use the [geemap](https://geemap.org) Python package to display the maps. Geemap enables users to interactively explore and visualize Earth Engine datasets within a Jupyter-based environment with minimal coding. To learn more about geemap, check out https://geemap.org.

Following are the steps to model AGBD.

# Initialize and Authenticate Earth Engine
To get started with Google Earth Engine (GEE), you need to initialize and authenticate the Earth Engine API. Follow these steps.


First, import the Earth Engine API by importing the ee module into your Python environment. This module allows you to interact with the Earth Engine platform.


In [1]:
# Import the API
import ee

# Import the geemap library
import geemap

Next, initialize the Earth Engine API. You must initialize the API to use Earth Engine functionalities. This involves authenticating your session and initializing the library. When you run the ee.Initialize() command for the first time, you might be prompted to authenticate your session. This will open a web browser window where you need to log in with your Google account and grant Earth Engine access.

In [2]:
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize(project='ee-kamusoko-test') # Change to your EE project

## Define the boundary
Next, select your project boundary.

In [4]:
# Load the GeoBoundaries administrative boundaries datasets
admin2 = ee.FeatureCollection("projects/sat-io/open-datasets/geoboundaries/CGAZ_ADM2")

# Filter the admin2 collection for Colombo (Sri Lanka).
# You might need to adjust the name according to the dataset.
boundary = admin2.filter(ee.Filter.eq('shapeName', 'Colombo District'))

# Check the count of administrative regions for Colombo
print('Colombo Admin2 count:', boundary.size().getInfo())

# Create an interactive map centered on Colombo (approximate center)
Map = geemap.Map(center=[6.9271, 79.8612], zoom=10)

# Add the Colombo admin2 layer to the map
Map.addLayer(boundary, {'color': 'green'}, 'Colombo Admin2')

# Display the map
Map

Colombo Admin2 count: 1


Map(center=[6.9271, 79.8612], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDat…

## Create Sentinel-2 composite
The sentinel-2 mission offers a wide-swath, high-resolution, multispectral imaging capability with a global 5-day revisit frequency. The Sentinel-2 Multispectral Instrument (MSI) has 13 spectral bands, providing a comprehensive view of the Earth's surface. These bands are distributed as four at 10 meters, six at 20 meters, and three at 60 meters spatial resolution. For more detailed information about the Sentinel-2 mission, please visit https://sentinel.esa.int/web/sentinel/missions/sentinel-2.


In [6]:
# Sentinel-2 SR data (Harmonized)
s2 = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')

# Cloud masking function using SCL band
def mask_s2clouds(image):
    scl = image.select('SCL')
    mask = scl.neq(8).And(scl.neq(9)).And(scl.neq(10)).And(scl.neq(11))
    return image.updateMask(mask).divide(10000)

# Filter and preprocess Sentinel-2 data
S2 = (s2.filterBounds(boundary)
      .filterDate('2024-03-01', '2024-06-30')
      .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10))
      .map(mask_s2clouds)
      .select(['B2','B3','B4','B5','B6','B7','B8','B11','B12']))

# Create a median composite
composite = S2.median().clip(boundary)

# Initialize our map.
map1 = geemap.Map()
map1.centerObject(boundary, 12)

# Add the boundary layer to the map
# map1.addLayer(ee.Image().paint(boundary, 1, 2), boundary_style, "Bulawayo")

# Add the composite image to the map with specified display settings.
map1.addLayer(composite, {'bands': ['B11', 'B8', 'B3'], 'min': 0, 'max': 0.3}, 'Sentinel-2 Composite')

# Display the map with layer control.
map1.addLayerControl()
map1

Map(center=[6.869336990108315, 80.02018121850574], controls=(WidgetControl(options=['position', 'transparent_b…

## Compute spectral indices
In this tutorial, we will utilize the enhanced vegetation index (EVI), normalized difference water index (NDWI), and normalized difference vegetation index (NDVI). Next, calculate EVI, NDWI, and NDVI.

In [7]:
# Compute EVI
evi = composite.expression(
    '2.5 * ((NIR - RED) / (NIR + 6*RED - 7.5*BLUE + 1))', {
        'NIR': composite.select('B8'),
        'RED': composite.select('B4'),
        'BLUE': composite.select('B2')
    }).rename('EVI')

# Compute NDWI
ndwi = composite.expression(
    '(GREEN - NIR) / (GREEN + NIR)', {
        'GREEN': composite.select('B3'),
        'NIR': composite.select('B8')
    }).rename('NDWI')

# Compute NDVI
ndvi = composite.expression(
    '(NIR - RED) / (NIR + RED)', {
        'NIR': composite.select('B8'),
        'RED': composite.select('B4')
    }).rename('NDVI')

# Initialize our map.
map2 = geemap.Map()
map2.centerObject(boundary, 12)

# Add the NDVI and RESI layers to the map.
map2.addLayer(evi, {'min': -1, 'max': 1, 'palette': ['blue', 'white', 'green']}, 'EVI')
map2.addLayer(ndwi, {'min': -1, 'max': 1, 'palette': ['blue', 'white', 'green']}, 'NDWI')
map2.addLayer(ndvi, {'min': -1, 'max': 1, 'palette': ['blue', 'white', 'green']}, 'NDVI')

# Display the map with layer control.
map2.addLayerControl()
map2

Map(center=[6.869336990108315, 80.02018121850574], controls=(WidgetControl(options=['position', 'transparent_b…

## Global PALSAR-2/PALSAR Yearly Mosaic (version 2)
The global 25m PALSAR/PALSAR-2 mosaic is a seamless global SAR image created by mosaicking strips of SAR imagery from PALSAR/PALSAR-2. For each year and location, the strip data were selected through visual inspection of the browse mosaics available over the period, with those showing minimum response to surface moisture preferentially used. Only data from the target year have been used for each annual mosaic, and hence no gap-filling using data from previous years in case of gaps in the annual global coverage.


In [8]:
# Load the SAR dataset and filter by date for 2023.
dataset = (ee.ImageCollection('JAXA/ALOS/PALSAR/YEARLY/SAR_EPOCH')
           .filter(ee.Filter.date('2023-01-01', '2023-12-01')))

# Create a single SAR image by mosaicking the collection and clipping it to the boundary.
sar_image = dataset.mosaic().clip(boundary)

# Define visualization parameters for the HH and HV bands.
sar_vis = {'min': 0.0, 'max': 10000.0}

# Initialize our map.
map3 = geemap.Map()
map3.centerObject(boundary, 12)

# Normalize using a known range (0 to 10000).
sar_norm = sar_image.select(['HH', 'HV']).unitScale(0, 10000) \
                    .rename(['HH_norm', 'HV_norm'])

# Add the PALSAR datasets to the map.
# Display the HH polarization layer.
map3.addLayer(sar_image.select('HH'), sar_vis, 'SAR HH')
# Display the HV polarization layer.
map3.addLayer(sar_image.select('HV'), sar_vis, 'SAR HV')

# Visualize the normalized bands (stretched to 0–1).
norm_vis = {'min': 0, 'max': 1}
map3.addLayer(sar_norm.select('HH_norm'), norm_vis, 'Normalized HH')
map3.addLayer(sar_norm.select('HV_norm'), norm_vis, 'Normalized HV')

# Display the map with layer control.
map3.addLayerControl()
map3

Map(center=[6.869336990108315, 80.02018121850574], controls=(WidgetControl(options=['position', 'transparent_b…

# Prepare datasets for modeling
## Merge all predictor variables
Combine Sentinel-2 bands, spectral indices (NDVI, SAVI, RESI), and PALSAR HH and HV.

In [9]:
# Combine the Sentinel-2 composite with the computed indices and the normalized SAR bands.
composite_with_indices = composite.addBands([evi, ndwi, ndvi]) \
                                  .addBands(sar_norm)  # Add HH_norm and HV_norm

# Define the list of bands to include in the regression.
bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B11', 'B12',
         'EVI', 'NDWI', 'NDVI', 'HH_norm', 'HV_norm']

In [10]:
# Load ESA WorldCover 2020 data and clip to boundary
esa_landcover = ee.ImageCollection('ESA/WorldCover/v100').first()
esa_landcover_clipped = esa_landcover.clip(boundary)

# Prepare visualization parameters
visualization = {
    'bands': ['Map'],
}


Import the training data feature collection.


In [11]:
# Load the FeatureCollection
training_data1 = ee.FeatureCollection('projects/ee-kamusoko-test/assets/Col_training_data2')

# Convert 'Map' property from float to integer
def convert_map_to_int(feature):
    return feature.set('Map', ee.Number(feature.get('Map')).toInt())

training_data_int = training_data1.map(convert_map_to_int)

# Print to console to inspect attributes
#print('Feature Collection with Integer Map:', training_data_int.getInfo())

# Mapping Land Cover using Random Forests
## Extracting training samples from composite image
Next, let's perform preprocessing, where image features (e.g., spectral indices) are extracted based on known training locations to build a model. The sampleRegions() function extracts pixel values from composite_with_indices at the locations specified in training_data, where the collection=training_data argument provides the reference points for sampling. The scale=10 parameter sets the spatial resolution at which values are extracted, in this case, at a 10-meter resolution. After extraction, extractTA.size().getInfo() retrieves and prints the total number of extracted samples. The getInfo() function is used to fetch the value from Google Earth Engine, allowing it to be displayed in Python.


In [12]:
# Extracting training samples
extractTA = composite_with_indices.sampleRegions(
    collection=training_data_int,
    scale=10
)

Split training datasets into training and testing sets. Then check the size of the training and testing data.

In [13]:
# Split into training (80%) and testing (20%) sets
withRandom = extractTA.randomColumn(columnName='random', seed=42)
training_set = withRandom.filter(ee.Filter.lt('random', 0.8))
testing_set = withRandom.filter(ee.Filter.gte('random', 0.8))

# Print training and testing set sizes (optional)
print('Training Set Size:', training_set.size().getInfo())
print('Testing Set Size:', testing_set.size().getInfo())

Training Set Size: 7519
Testing Set Size: 1827


## Train the random forest model
Train a random forest (RF) model and classify land cover.

In [14]:
# Train Random Forest classifier
classifier = ee.Classifier.smileRandomForest(numberOfTrees=500).train(
    features=training_set,
    classProperty='Map',
    inputProperties=composite_with_indices.bandNames()
)

# Classify the composite
classified = composite_with_indices.classify(classifier)

Display the land cover map.

In [15]:
# Initialize the map
map4 = geemap.Map()
map4.centerObject(boundary, 12)

# Add Google Satellite Image as a basemap
map4.add_basemap('SATELLITE')

# Prepare visualization parameters
land_cover_vis = {
    'min': 10,
    'max': 70,
    'palette': [
        '006400',  # Tree cover
        'ffbb22',  # Shrubland
        'ffff4c',  # Grassland
        'f096ff',  # Cropland
        'fa0000',  # Built-up
        'b4b4b4',  # Bare/sparse vegetation
        '0064c8'   # Water
    ]
}

# Add land cover classification layer
map4.addLayer(classified, land_cover_vis, 'Land Cover Classification')

# Define land cover legend dictionary
legend_dict = {
    'Tree cover': '006400',
    'Shrubland': 'ffbb22',
    'Grassland': 'ffff4c',
    'Cropland': 'f096ff',
    'Built-up': 'fa0000',
    'Bare/sparse vegetation': 'b4b4b4',
    'Water': '0064c8'
}

# Add legend to the map
map4.add_legend(title="Land Cover Legend", legend_dict=legend_dict)

# Display the map
map4

Map(center=[6.869336990108315, 80.02018121850574], controls=(WidgetControl(options=['position', 'transparent_b…

# Check Model Performance
Next, we will evaluate the model's performance using overall accuracy and kappa coefficient.


In [16]:
# Classify the testing dataset
test_results = testing_set.classify(classifier)

# Compute the confusion matrix
confusion_matrix = test_results.errorMatrix('Map', 'classification')

# Print overall accuracy
overall_accuracy = confusion_matrix.accuracy().getInfo()
print('Overall Accuracy:', overall_accuracy)

Overall Accuracy: 0.6250684181718664
